In [59]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

# import helper functions
%run -i helper_functions.py

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

# Read files

In [2]:
train_features = pd.read_csv("./Data/selected_train.csv")
test_features = pd.read_csv("./Data/selected_test.csv")
embedding = pd.read_csv("./Data/embeddings_glove.csv")

In [3]:
cleaned_train = pd.read_csv("./Data/cleaned_train.csv")
test = pd.read_csv("./Data/test.csv")
test_labels = pd.read_csv("Data/test_labels.csv")
print(test_labels.shape)
test_labels.head()

(153164, 7)


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [4]:
# join text data and labels
test_labeled = pd.concat([test, test_labels.drop('id', axis=1)], axis=1)
masking = (test_labeled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]==-1).sum(axis=1)==0
test_labeled = test_labeled[masking].reset_index(drop=True)
test_cleaned = data_cleaning(test_labeled)

removing noise
further cleaning the text


# LSTM on clean text

## Define parameters

In [5]:
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

## Tokenize and sequence train and test clean texts

In [40]:

train_y = train_features[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

train_x = cleaned_train['clean_text']

test_x = test_cleaned['clean_text']


# Vectorize text + Prepare GloVe Embedding
tokenizer = Tokenizer(num_words=max_features, lower=True)
tokenizer.fit_on_texts(list(train_x))

train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

train_x = pad_sequences(train_x, maxlen=maxlen)
test_x = pad_sequences(test_x, maxlen=maxlen)
test_y = test_cleaned[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values


## Embedding matrix according to Glove (How to extract from ours?)

In [7]:
EMBEDDING_FILE = "Data/glove.6B.50d.txt"
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

In [9]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

## Hyperparameter tuning

### Define inital model

In [30]:
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
def build_model(hp):          #hp means hyper parameters
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
    x = GlobalMaxPool1D()(x)
    #providing range for number of neurons in a hidden layer
    x = Dense(units=hp.Int('num_of_neurons1',min_value=20,max_value=60,step=10), activation="relu")(x)
    x = Dropout(0.1)(x)
    #output layer
    x = Dense(6, activation="sigmoid")(x)
    #compiling the model
    model = Model(inputs=inp, outputs=x)
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4])),loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [31]:
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='tuner',
                     project_name='LSTM')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

INFO:tensorflow:Reloading Tuner from tuner/LSTM/tuner0.json


### Search

In [32]:
tuner.search(train_x,train_y,epochs=5, validation_split=0.1, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('num_of_neurons1')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 24 Complete [01h 29m 52s]
val_accuracy: 0.9939842224121094

Best val_accuracy So Far: 0.9940468668937683
Total elapsed time: 12h 56m 53s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 20 and the optimal learning rate for the optimizer
is 0.001.



### Build model with best params

In [35]:
model_tuned = tuner.hypermodel.build(best_hps)
history = model_tuned.fit(train_x,train_y, epochs=5, validation_split=0.1)

Epoch 1/5
4488/4488 [==============================] - 456s 100ms/step - loss: 0.0647 - accuracy: 0.9261 - val_loss: 0.0504 - val_accuracy: 0.9940
Epoch 2/5
4488/4488 [==============================] - 423s 94ms/step - loss: 0.0473 - accuracy: 0.9898 - val_loss: 0.0480 - val_accuracy: 0.9940
Epoch 3/5
4488/4488 [==============================] - 468s 104ms/step - loss: 0.0431 - accuracy: 0.9897 - val_loss: 0.0471 - val_accuracy: 0.9939
Epoch 4/5
4488/4488 [==============================] - 438s 98ms/step - loss: 0.0397 - accuracy: 0.9874 - val_loss: 0.0477 - val_accuracy: 0.9940
Epoch 5/5
4488/4488 [==============================] - 478s 107ms/step - loss: 0.0366 - accuracy: 0.9844 - val_loss: 0.0513 - val_accuracy: 0.9937


In [36]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(train_x,train_y, epochs=best_epoch, validation_split=0.1)

Best epoch: 1
4488/4488 [==============================] - 506s 111ms/step - loss: 0.0647 - accuracy: 0.9080 - val_loss: 0.0496 - val_accuracy: 0.9940


### Evaluation model result

In [41]:
hypermodel.evaluate(test_x, test_y)

2000/2000 [==============================] - 31s 15ms/step - loss: 0.0719 - accuracy: 0.9976


### Prediction on best model

In [44]:
predictions = hypermodel.predict(test_x)

2000/2000 [==============================] - 34s 16ms/step


### Evaluation scores

In [60]:
get_evaluation_score(np.argmax(test_y, axis=1), np.argmax(predictions, axis=1)) 

Accuracy score:  0.9975772921941918
Precision score:  0.9952227503172172
Recall score:  0.9975772921941918
F1 score:  0.9963986302813638
Mean squared error:  0.02788458532620588
Mean absolute error:  0.007783925724467786


/Users/yco/opt/anaconda3/envs/BT4222/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [48]:
# Prediction
batch_size = 32
epochs = 2

model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1)


Epoch 1/2
4488/4488 [==============================] - 1357s 302ms/step - loss: 0.0556 - accuracy: 0.9270 - val_loss: 0.0476 - val_accuracy: 0.9939
Epoch 2/2
4488/4488 [==============================] - 1226s 273ms/step - loss: 0.0431 - accuracy: 0.9668 - val_loss: 0.0447 - val_accuracy: 0.9819


In [ ]:
predictions = model.predict(test_x, batch_size=batch_size, verbose=1)